In [1]:
from database.database.ask_db import *
from database.operations.collect_fens import get_new_games_links, get_fens_from_games, get_fens_from_games
from database.database.models import to_dict
import chess

/home/jon/anaconda3/envs/chessism_gpu/lib/python3.12/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [4]:
new_game_links = get_new_games_links(100)

In [6]:
new_game_links[0]

(3400625,)

In [5]:
fen_set_from_games = get_fens_from_games(new_game_links)

NameError: name 'get_one_game' is not defined

In [ ]:
get_only_new_fens = get_new_fens(fen_set_from_game)

In [23]:
a = get_one_game(valid[0][0])

In [25]:
def get_fens_from_games(moves):
    board = chess.Board() # Initialize a standard chess board
    fens_sequence = []
    for ind, move in enumerate(moves):
        assert move['n_move'] == ind+1
        n_move = move['n_move']
        white_move_san = move['white_move']
        black_move_san = move['black_move']
        current_fens = {'n_move': n_move}
        
        try:
            move_obj_white = board.parse_san(white_move_san)
            board.push(move_obj_white)
            fens_sequence.append(board.fen())
        except ValueError as e:
            print(f"Error applying White's move '{white_move_san}' at move number {n_move}: {e}")
            return fens_sequence # Stop processing on error
        except chess.InvalidMoveError as e:
            print(f"Invalid White's move '{white_move_san}' at move number {n_move}: {e}")
            return fens_sequence # Stop processing on error


        # 2. Apply Black's move and get FEN
        try:
            move_obj_black = board.parse_san(black_move_san)
            board.push(move_obj_black)
            fens_sequence.append(board.fen())
        except ValueError as e:
            print(f"Error applying Black's move '{black_move_san}' at move number {n_move}: {e}")
            return fens_sequence # Stop processing on error
        except chess.InvalidMoveError as e:
            print(f"Invalid Black's move '{black_move_san}' at move number {n_move}: {e}")
            return fens_sequence # Stop processing on error

        #fens_sequence.append(current_fens)
    return fens_sequence

In [26]:
a = get_fens_from_games(a)

In [27]:
a

['rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR b KQkq - 0 1',
 'rnbqkbnr/pp1ppppp/2p5/8/3P4/8/PPP1PPPP/RNBQKBNR w KQkq - 0 2',
 'rnbqkbnr/pp1ppppp/2p5/8/2PP4/8/PP2PPPP/RNBQKBNR b KQkq - 0 2',
 'rnbqkbnr/pp2pppp/2p5/3p4/2PP4/8/PP2PPPP/RNBQKBNR w KQkq - 0 3',
 'rnbqkbnr/pp2pppp/2p5/3p4/2PP4/5N2/PP2PPPP/RNBQKB1R b KQkq - 1 3',
 'rnbqkb1r/pp2pppp/2p2n2/3p4/2PP4/5N2/PP2PPPP/RNBQKB1R w KQkq - 2 4',
 'rnbqkb1r/pp2pppp/2p2n2/3p4/2PP4/2N2N2/PP2PPPP/R1BQKB1R b KQkq - 3 4',
 'rnbqkb1r/pp3ppp/2p1pn2/3p4/2PP4/2N2N2/PP2PPPP/R1BQKB1R w KQkq - 0 5',
 'rnbqkb1r/pp3ppp/2p1pn2/3P4/3P4/2N2N2/PP2PPPP/R1BQKB1R b KQkq - 0 5',
 'rnbqkb1r/pp3ppp/2p2n2/3p4/3P4/2N2N2/PP2PPPP/R1BQKB1R w KQkq - 0 6',
 'rnbqkb1r/pp3ppp/2p2n2/3p2B1/3P4/2N2N2/PP2PPPP/R2QKB1R b KQkq - 1 6',
 'rnbqk2r/pp3ppp/2pb1n2/3p2B1/3P4/2N2N2/PP2PPPP/R2QKB1R w KQkq - 2 7',
 'rnbqk2r/pp3ppp/2pb1n2/3p2B1/3P4/2N2N2/PPQ1PPPP/R3KB1R b KQkq - 3 7',
 'rnbqk2r/pp3pp1/2pb1n1p/3p2B1/3P4/2N2N2/PPQ1PPPP/R3KB1R w KQkq - 0 8',
 'rnbqk2r/pp3pp1/2pb1n1p/3p4/3P3B

In [ ]:
def get_fens_for_game_moves(moves_list: list[dict]) -> list[dict]:
    """
    Generates FENs for each half-move in a list of chess moves.

    Args:
        moves_list: A list of dictionaries, where each dictionary has
                    'n_move', 'white_move' (SAN), and 'black_move' (SAN).

    Returns:
        A list of dictionaries, each containing:
        - 'n_move': The move number.
        - 'white_fen': The FEN string after White's move.
        - 'black_fen': The FEN string after Black's move.
        Returns an empty list or stops processing if an invalid move is encountered.
    """
    board = chess.Board() # Initialize a standard chess board
    fens_sequence = []

    for move_entry in moves_list:
        n_move = move_entry['n_move']
        white_move_san = move_entry['white_move']
        black_move_san = move_entry['black_move']

        current_fens = {'n_move': n_move}

        # 1. Apply White's move and get FEN
        try:
            move_obj_white = board.parse_san(white_move_san)
            board.push(move_obj_white)
            current_fens['white_fen'] = board.fen()
        except ValueError as e:
            print(f"Error applying White's move '{white_move_san}' at move number {n_move}: {e}")
            # If a move is invalid, the sequence is broken. You might want to
            # return what you have so far or raise an error depending on your needs.
            return fens_sequence # Stop processing on error
        except chess.InvalidMoveError as e:
            print(f"Invalid White's move '{white_move_san}' at move number {n_move}: {e}")
            return fens_sequence # Stop processing on error


        # 2. Apply Black's move and get FEN
        try:
            move_obj_black = board.parse_san(black_move_san)
            board.push(move_obj_black)
            current_fens['black_fen'] = board.fen()
        except ValueError as e:
            print(f"Error applying Black's move '{black_move_san}' at move number {n_move}: {e}")
            return fens_sequence # Stop processing on error
        except chess.InvalidMoveError as e:
            print(f"Invalid Black's move '{black_move_san}' at move number {n_move}: {e}")
            return fens_sequence # Stop processing on error

        fens_sequence.append(current_fens)

    return fens_sequence

# Get the FENs for your list of moves
game_fens_result = get_fens_for_game_moves(moves_data)

# Print the results
for entry in game_fens_result:
    print(f"Move {entry['n_move']}:")
    print(f"  White FEN: {entry['white_fen']}")
    print(f"  Black FEN: {entry['black_fen']}")

# You can also verify the final board state if needed
if game_fens_result:
    print(f"\nFinal FEN after all moves: {game_fens_result[-1]['black_fen']}")

In [14]:
a

[{'n_move': 1, 'white_move': 'd4', 'black_move': 'c6'},
 {'n_move': 2, 'white_move': 'c4', 'black_move': 'd5'},
 {'n_move': 3, 'white_move': 'Nf3', 'black_move': 'Nf6'},
 {'n_move': 4, 'white_move': 'Nc3', 'black_move': 'e6'},
 {'n_move': 5, 'white_move': 'cxd5', 'black_move': 'exd5'},
 {'n_move': 6, 'white_move': 'Bg5', 'black_move': 'Bd6'},
 {'n_move': 7, 'white_move': 'Qc2', 'black_move': 'h6'},
 {'n_move': 8, 'white_move': 'Bh4', 'black_move': 'O-O'},
 {'n_move': 9, 'white_move': 'e3', 'black_move': 'Bg4'},
 {'n_move': 10, 'white_move': 'Bd3', 'black_move': 'Nbd7'},
 {'n_move': 11, 'white_move': 'h3', 'black_move': 'Be6'},
 {'n_move': 12, 'white_move': 'O-O', 'black_move': 'Qc7'},
 {'n_move': 13, 'white_move': 'Rad1', 'black_move': 'g5'},
 {'n_move': 14, 'white_move': 'Bxg5', 'black_move': 'hxg5'},
 {'n_move': 15, 'white_move': 'Nxg5', 'black_move': 'Kg7'},
 {'n_move': 16, 'white_move': 'Bf5', 'black_move': 'Bxf5'},
 {'n_move': 17, 'white_move': 'Qxf5', 'black_move': 'Rh8'},
 {'n_m